<br>
<br>
<br>
<br>

# Module 3: Dimensionality Reduction & Feature Selection
<br>
<br>
<br>

## "Simpler" is Better

- Machine learning models that are overly complex can be difficult, if not impossible, to interpret. 

- Complex models are computationally expensive

- A great deal of model complexity can be eliminated by reducing the number of explanatory variables employed within a model. This is done via __Dimensionality Reduction__ and __Feature Selection__ techniques.


## Dimensionality Reduction

* "Dimensionality" refers to the number of potential explanatory variables (a.k.a., "features") within a data set.


* For a very large number of features relative to the number of observations, some machine learning algorithms will fail to produce effective models, particularly those which are based on similarity metrics (e.g., K-nearest neighbors, clustering, etc.). 


* We can reduce the number of __numeric__ features to be used within a model via the application of linear algebra techniques (e.g., Principal Components Analysis (PCA); Singular Value Decomposition (SVD)). These methods allow us to derive orthogonal features from a non-orthogonal collection of __numeric__ features.


## Principal Components Analysis (PCA)

- PCA uses an orthogonal transformation to convert a set of possibly correlated __numeric__ features into a set of values of linearly uncorrelated (a.k.a., __orthogonal__) features known as __principal components__. These new features replace the old ones.  The principal components themselves are assessed by ranking them in order of the amount of variance in the data they explain. When used within a machine learning context, we select the most relevant principal components (i.e., those that explain the greatest amount of variance). We want to retain the principal components that cumulatively explain at least x% of the variability in the data. 

One cannot use PCA on categorical features, even when they have been converted to binary "one-hot" dummy variables. If you want to reduce the dimensionality of categorical data you should instead rely on the use of __feature selection__ techniques.

Principal components are extremely difficult to interpret, so one must be careful when using such a model.  

[Here](https://towardsdatascience.com/pca-and-svd-explained-with-numpy-5d13b0d2a4d8) is a good simple example.


__sklearn.decomposition.PCA()__ function can be used to identify the principal components of a data set. [Link](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html)


[Here](https://stackoverflow.com/questions/32194967/how-to-do-pca-and-svm-for-classification-in-python) is an example adapted from  a stackoverflow question: 

In [ ]:
# load numpy + 'datasets'. We'll use the "iris" data that is provided with sklearn
import numpy as np
from sklearn import datasets
import warnings
warnings.filterwarnings('ignore')

# load PCA + SVM classifier ("SVC") + cross validation functions
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn import model_selection

# load the iris dataset
iris = datasets.load_iris()

# what is the dimensionality of the data?
iris.data.shape

(150, 4)

In [ ]:
# display the first row of the array
iris.data[0]

array([5.1, 3.5, 1.4, 0.2])

As shown above, each of the features contains floating point numbers. Since all of the features are numeric we can apply PCA to the data set in an attempt to reduce its dimensionality.

In [ ]:
# assign the explanatory variables to a Python object
X = iris.data

# assign the response variable to a Python object
y = iris.target

# split the data into training + testing subsets
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.4, random_state=0)

# create an instance of a PCA model +
# set the number of components you want to retain
pca = PCA(n_components=2)

# apply the PCA function to the training data
pca.fit(X_train)

# display the explained variance ratio for the principal components we've derived from the data
print(pca.explained_variance_ratio_)

[0.9348581  0.04635375]


The application of PCA to the data set has yielded two principal components, the first of which explains 93.485% of the variance in the iris data. The second principal component explains an additional 4.635% of the variance in the data. Therefore, the two principal components explain a total of more than 98% of the variance in the data set.

In [ ]:
# display the singular values associated with the 2 principal components
print(pca.singular_values_)

[20.62747613  4.59320324]


In [ ]:
# now apply the results of the PCA to the training data to transform it into 2 principal components per observation
X_t_train = pca.transform(X_train)

# apply the results of the PCA to the testing data to transform it into 2 principal components per observation
X_t_test = pca.transform(X_test)

# create an instance of an SVM classifier
clf = SVC()

# fit the SVM classifier to the transformed training data + the response data
clf.fit(X_t_train, y_train)

# check the accuracy of the SVM classifier using the transformed explanatory variables + the response variable
print ('score', clf.score(X_t_test, y_test) )

score 0.9333333333333333


Our SVM classifier has achieved an accuracy score of 93.3% when applied to the two principal components we derived from the iris data. 

In [ ]:
# if desired, generate predictions from the SVM classifier for the transformed testing data
print ('pred label', clf.predict(X_t_test) )

pred label [2 1 0 2 0 2 0 1 1 1 2 1 1 1 1 0 1 1 0 0 2 1 0 0 1 0 0 1 1 0 2 1 0 1 2 1 0
 2 1 1 2 0 2 0 0 1 2 2 2 2 1 2 1 1 2 2 1 2 1 2]


## Feature Selection

We can determine which attributes/features to include via a variety of threshold "filters" (e.g., exclude all variables whose variance falls below a certain value; if two or more variables are highly correlated with one another, choose one to use within the model and exclude the others; etc.)

Some of the most commonly used feature selection techniques include:

__Variance Threshholds__: Normalize the features you plan to use as explanatory variables, then calculate their variances. Features whose values show relatively little variance are much less likely to introduce valuable information within the context of a model, so they are strong candidates for exclusion. This choice is highly subjective.


__Correlation Thresholds__: Remove features that are highly correlated with other features. As with variance thresholds, the choice of a correlation value to use as a threshold is highly subjectve.


__Forward Selection__: Incrementally add features to a model one at a time until model performance no longer improves. A common approach to Forward Stepwise Search is to begin your modeling using the explanatory variable that is most highly correlated with the response variable, then sequentially add additional explanatory variables in decreasing order of their correlation with the response variable. The general algorithm is as follows:

- Start with the null model, a model containing an intercept but no predictors.
- Fit a simple linear regression model to each individual explanatory variable and then add to the null model that variable resulting in the lowest residual sum of squares value ("RSS").
- Add to that model the variable that results in the lowest RSS amongst all two-variable models.
- The algorithm continues until some stopping rule is satisfied (i.e. all remaining variables have a p-value greater than some
threshold).

[Here](https://towardsdatascience.com/feature-selection-using-wrapper-methods-in-python-f0d352b346f) is an example of this.


__Backward Selection__: Used in regression modeling. Start your modeling process using all of the explanatory variables you believe to be appropriate, then sequentially remove variables one at a time until model performance starts to substantively degrade. The general algorithm is as follows:

- Begin with all variables in the model.
- Remove the variable with the largest p-value (i.e. least statistically significant).
- The new model is fit, and the variable with the largest p-value is removed.
- The algorithm continues until a stopping rule is reached (e.g., the p-values of all variables are <= 0.05)

[Here[(https://towardsdatascience.com/feature-selection-using-wrapper-methods-in-python-f0d352b346f) is an example of this.


__Variance Inflation Factors (VIF)__: A regression-specific metric that can use to help us select features for inclusion within a regression algorithm. A VIF value provides an indication of the presence of multicolinearity amongst the explanatory variables used for regression modeling. VIF is calculated by regressing an explanatory variable against every other available explanatory variable. In general, if your VIF calculations for a feature result in a VIF > 5, that feature is a strong candidate for removal from the model: rerun the model without the variable and check to see whether the model's performance has improved. If so, exclude the feature from the model. If not, leave the feature in the model. [Example link](https://www.statisticshowto.datasciencecentral.com/variance-inflation-factor/)



# Assignment 3



Take the data and apply feature selection  and/or dimensionality reduction techniques to identify the explanatory variables to be included within a linear  regression model that predicts automobile gas mileage. The data set you will be using is sourced from the UC  Irvine machine learning archive:  

• https://archive.ics.uci.edu/ml/datasets/Automobile 

The data set is comprised of 205 observations and 26 attributes. Check the UCI web page for further  details. The city-mpg variable is our response variable for your regression model.  Apply your feature selection / dimensionality reduction expertise to the remaining 25  attributes to identify the explanatory variables that you believe will be most useful when  included in a linear regression model that estimates city-mpg.


<br>
</br>

###Steps

1. Load the data into a dataframe.
2. Perform some basic exploratory data analysis (EDA). Include  any insights you are able to derive from your statistical analysis of the attributes and the accompanying  exploratory graphics you create (e.g., bar plots, box plots, histograms, line plots, etc.). Identify preliminary predictive inferences, e.g., do any of the explanatory variables appear  to be relatively more “predictive” of the response variable? Make sure to select  appropriate statistical metrics to be calculated + which types of exploratory graphics to make use of.  Your goal should be to provide an EDA that is thorough and succinct without it being so detailed that a  reader will lose interest in it. Your introduction should include a summary of the problem + explain the steps you plan to take to address the  problem. Your EDA should include any preliminary predictive inferences as well as explanation. 
3. Use feature selection and dimensionality reduction to the  25 candidate explanatory variables to identify variables that you believe will prove to be relatively  useful within the required linear regression model. Your work should reflect some of the knowledge you have gained via your EDA work. While selecting your features, be sure to consider the  tradeoff between model performance and model simplification, e.g., if you are reducing the complexity  of your model, are you sacrificing too much in the way of accuracy (or some other performance  measure)? Be sure to  include an explanatory narrative that justifies your decision making process. 
4. Train/cross validate your model and report on its accuracy. 

